# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
cities_data_to_load = "C:/Users/paul_/workspace/python-api-challenge/WeatherPy/output_data_weather/cities.csv"
cities_data = pd.read_csv(cities_data_to_load)
cities_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Alugan,12.22,125.48,84.72,80,100,13.18,PH,1603252574
1,1,Walvis Bay,-22.96,14.51,57.99,82,0,10.29,NaN,1603252574
2,2,San Policarpo,12.18,125.51,84.67,79,100,14.12,PH,1603252512
3,3,Hilo,19.73,-155.09,82.40,58,90,6.93,US,1603252297
4,4,Jamestown,42.10,-79.24,48.99,100,90,5.82,US,1603252574


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
gmaps.configure(api_key=g_key)
locations = cities_data[["Lat", "Lng"]].astype(float)
humidities = cities_data["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidities, 
                                 dissipating=False, max_intensity=100,
                                point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
perfect_cities = cities_data.loc[(cities_data['Max Temp']<=85)&(cities_data['Max Temp']>=75)&(cities_data["Wind Speed"]<=10)&(cities_data["Humidity"]<=60)&(cities_data["Cloudiness"]==0)]
perfect_cities

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,Nizwá,22.93,57.53,81.86,11,0,4.23,OM,1603252587
56,56,Ankazoabo,-22.28,44.52,75.79,40,0,5.77,MG,1603252588
167,167,Sur,22.57,59.53,81.39,50,0,6.76,OM,1603252614
198,198,Aswān,24.09,32.91,76.84,24,0,8.66,EG,1603252622
224,224,Araouane,18.90,-3.53,82.17,19,0,8.95,ML,1603252628
336,336,Agadez,19.75,10.25,77.02,18,0,9.71,NE,1603252654
429,429,Umluj,25.02,37.27,83.10,32,0,4.27,SA,1603252675
431,431,Swan Hill,-35.34,143.55,77.59,28,0,7.27,AU,1603252675
504,504,Adrar,20.50,-10.07,80.60,19,0,4.90,MR,1603252692
512,512,Miandrivazo,-19.52,45.47,82.26,52,0,0.22,MG,1603252694


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
hotel_df = perfect_cities
hotel_df["Hotel"] = ''
hotel_df

C:\Users\paul_\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
54,54,Nizwá,22.93,57.53,81.86,11,0,4.23,OM,1603252587,
56,56,Ankazoabo,-22.28,44.52,75.79,40,0,5.77,MG,1603252588,
167,167,Sur,22.57,59.53,81.39,50,0,6.76,OM,1603252614,
198,198,Aswān,24.09,32.91,76.84,24,0,8.66,EG,1603252622,
224,224,Araouane,18.90,-3.53,82.17,19,0,8.95,ML,1603252628,
336,336,Agadez,19.75,10.25,77.02,18,0,9.71,NE,1603252654,
429,429,Umluj,25.02,37.27,83.10,32,0,4.27,SA,1603252675,
431,431,Swan Hill,-35.34,143.55,77.59,28,0,7.27,AU,1603252675,
504,504,Adrar,20.50,-10.07,80.60,19,0,4.90,MR,1603252692,
512,512,Miandrivazo,-19.52,45.47,82.26,52,0,0.22,MG,1603252694,


In [41]:
#Nearest Restaurants



base_url = "https://maps.googleapis.com/maps/key/place/nearbysearch/json"
params = {"keyword": "lodging",
         "radius": 5000,
         "type": "hotel",
         "key": g_key,}

city_coords = perfect_cities[['Lat', 'Lng']]

counter = 0

for index, row in perfect_cities.iterrows():
    
    perfect_city = row["City"]
    counter = counter + 1
    if counter >= 2:
        break
    if counter <= 15:
    
        time.sleep(1)
        lat = row["Lat"]
        lng = row["Lng"]
        params["city_coords"] = f'{lat}, {lng}'
      
    try:
        
        response = requests.get(base_url, params=params).json()
        perfect_cities.loc[index, "Hotel"] = response['results'][0]['name']
        #print(response)
        
    except:
        pass

perfect_cities

C:\Users\paul_\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
54,54,Nizwá,22.93,57.53,81.86,11,0,4.23,OM,1603252587,
56,56,Ankazoabo,-22.28,44.52,75.79,40,0,5.77,MG,1603252588,
167,167,Sur,22.57,59.53,81.39,50,0,6.76,OM,1603252614,
198,198,Aswān,24.09,32.91,76.84,24,0,8.66,EG,1603252622,
224,224,Araouane,18.90,-3.53,82.17,19,0,8.95,ML,1603252628,
336,336,Agadez,19.75,10.25,77.02,18,0,9.71,NE,1603252654,
429,429,Umluj,25.02,37.27,83.10,32,0,4.27,SA,1603252675,
431,431,Swan Hill,-35.34,143.55,77.59,28,0,7.27,AU,1603252675,
504,504,Adrar,20.50,-10.07,80.60,19,0,4.90,MR,1603252692,
512,512,Miandrivazo,-19.52,45.47,82.26,52,0,0.22,MG,1603252694,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
